In [27]:
import time
import os
import sys
import pymysql
import pandas as pd
from tqdm import tqdm

In [28]:
# connect to the huawei database
conn = pymysql.connect(
    host='123.60.177.92',  
    user='root', 
    password='Hzk2023@',  
    database='bridge2' 
)

if conn:
    print("connect to the database successfully")
else:
    print("connect to the database failed")
    sys.exit(1)

connect to the database successfully


In [29]:
query_tables = "SHOW TABLES LIKE 'table%'"
tables_df = pd.read_sql(query_tables, conn)
table_names = tables_df.values.flatten()
table_names

array(['table_20240517', 'table_20240518', 'table_20240519',
       'table_20240520', 'table_20240521', 'table_20240522',
       'table_20240523', 'table_20240524', 'table_20240529',
       'table_20240530', 'table_20240531', 'table_20240601',
       'table_20240602', 'table_20240603', 'table_20240604',
       'table_20240605', 'table_data'], dtype=object)

In [30]:
output_dir = 'data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [31]:
for table_name in tqdm(table_names, desc="Processing tables"):
    query = f"SELECT video_record_time, test_point_code, x, y FROM {table_name}"
    chunksize = 10000
    chunk_iter = pd.read_sql(query, conn, chunksize=chunksize)
    
    for df_chunk in tqdm(chunk_iter, desc=f"Reading chunks for {table_name}", leave=False):
        df_chunk['video_record_time'] = pd.to_datetime(df_chunk['video_record_time'])
        
        grouped = df_chunk.groupby('test_point_code')
        
        for test_point_code, group in tqdm(grouped, desc=f"Processing test points in {table_name}", leave=False):
            folder_path = os.path.join(output_dir, str(test_point_code))
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            
            group['date'] = group['video_record_time'].dt.date
            date_groups = group.groupby('date')
            
            for date, date_group in tqdm(date_groups, desc=f"Processing dates for {test_point_code}", leave=False):
                csv_filename = f"{test_point_code}_{date}.csv"
                csv_filepath = os.path.join(folder_path, csv_filename)
                
                if os.path.exists(csv_filepath):
                    date_group[['video_record_time', 'test_point_code', 'x', 'y']].to_csv(csv_filepath, mode='a', header=False, index=False)
                else:
                    date_group[['video_record_time', 'test_point_code', 'x', 'y']].to_csv(csv_filepath, index=False)

conn.close()

Processing tables:   0%|          | 0/17 [00:26<?, ?it/s]


KeyboardInterrupt: 